# Toxic Comment Challenge
* This is a very well known "Toxic Comment Challenge" from Kaggle.
* This will be challenging but considering you have experience with that Trump Tweets thing, I'm confident you'll pull it off.

# Data about Data 

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:
* *`toxic`*
* *`severe_toxic`*
* *`obscene`*
* *`threat`*
* *`insult`*
* *`identity_hate`*


* **train.csv** - the training set, contains comments with their binary labels
* **test.csv** - the test set, you must predict the toxicity probabilities for these comments. To deter hand labeling, the test set contains some comments which are not included in scoring.
* **sample_submission.csv** - a sample submission file in the correct format
* **test_labels.csv** - labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)

___

***Things to do***
- Perform the data pre-processing.
- Perform EDA
- If there's an empty `Comment`, drop it.
- Remove punctuation
- Convert a collection of raw documents to a matrix of TF-IDF features using sklern TfidVectorizer
- Make a sparse matrix with required data for training and testing set using the sparse.hstack() method
- Create an empty(np.zeros) array "preds" of test size
- Fit a logisticRegression model
- Using the model.predict_proba() of LogisticRegression, calculate the probability and add it in the prediction array.
- Make the pred array, a pandas dataframe and set column names.
 
***What will be new***
- Almost everything is new. 
 
***What will be tricky***
- TfidVectorizer would be tricky, but you can refer to this sklearn documentation [HERE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [6]:
import numpy as np 
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy import sparse

In [9]:
# Sample output

pd.read_csv('sample_submission.csv').head(5)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
